### This python program web-scraps expedia website to find hotels per search criteria provided in a CSV file and sends email to you with the price details. It also creates a spreadsheet with all available details so you can track the price changes over time. 

In [47]:
# We will use Selenium package for web scraping.
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

In [48]:
import pandas as pd

In [49]:
import time
import datetime

In [50]:
# Library for building and sending email
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

In [51]:
# I will be using Chrome browser for opening Expedia site. 
# If you are using some other browser, you will need the respective driver. 
# Some popular ones are GeckoDriver for firefox, EdgeDriver for Microsoft Edge, InternerExplorerDriver for IE, 
# SafariDriver for Safari browser and so on.
# You can download the chromedriver from https://sites.google.com/a/chromium.org/chromedriver/downloads
chromedriver = "C:\\Users\\ManishBahety\\Downloads\\chromedriver" # this is where I have the driver
browser = webdriver.Chrome(chromedriver)

In [52]:
# Define email credentials to use for sending emails. I have put the email id and password in a single row two column csv file 
# to avoid hardcoding in the code. You can put the information in a secure place and read it here in username and password variable. 
# My credentials.csv CSV file has just one line containing email id and password - xxxxxxxxx@yahoo.com,pwdstring
import csv

with open('credentials.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    for row in readCSV:
        username = row[0]
        password = row[1]

    print(username)
    print(password)


lansa_user01@yahoo.com
 ready2go@lansa


In [53]:
# Let's now build functions that will help us do web scraping of the expedia pages. We will have several functions. 

In [54]:
def goingto_hotel_chooser(hotel_name_city):
    going_to = browser.find_element_by_xpath("//input[@id='hotel-destination-hp-hotel']")
    time.sleep(3)
    going_to.clear()
    time.sleep(3)
    going_to.send_keys('  ' + hotel_name_city)
    time.sleep(3)
    first_item = browser.find_element_by_xpath("//a[@id='aria-option-0']")
    time.sleep(3)
    first_item.click()

In [55]:
def checkin_date_chooser(month, day, year):
    checkin_date_button = browser.find_element_by_xpath("//input[@id='hotel-checkin-hp-hotel']")
    checkin_date_button.clear()
    checkin_date_button.send_keys(month + '/' + day + '/' + year)

In [56]:
def checkout_date_chooser(month, day, year):
    checkout_date_button = browser.find_element_by_xpath("//input[@id='hotel-checkout-hp-hotel']")
    for i in range(11):
        checkout_date_button.send_keys(Keys.BACKSPACE)
    checkout_date_button.send_keys(month + '/' + day + '/' + year)

In [57]:
def search():
    search = browser.find_element_by_xpath("//button[@class='btn-primary btn-action  gcw-submit']")
    search.click()
    time.sleep(15)
    print('Search Results ready!')

In [58]:
# click the top search result. 
def click_top_result():
    try:
        top_result = browser.find_element_by_xpath("//a[@data-stid='open-hotel-information']")
        time.sleep(3)
        top_result.click()
    except Exception as e:
        pass

In [59]:
df = pd.DataFrame()
def compile_data():
    
    global df
    global room_type_list
    global room_area_list
    global price_list
    global feature1_list
    global feature2_list
    global feature3_list
    
    #room type
    room_type = browser.find_elements_by_xpath("//h2[@class='uitk-type-heading-500']")
    room_type_list = [value.text for value in room_type]
    print(room_type_list)
    
    #price
    price = browser.find_elements_by_xpath("//span[@class='content-hotel-lead-price--a11y']")
    price_list = [value.text for value in price]
    print(price_list)
    
    #room area
    room_area = browser.find_elements_by_xpath("//div[@class='room__size uitk-type-300 pwa-theme--grey-700 all-b-padding-two content']")
    room_area_list = [value.text for value in room_area]
    print(room_area_list)
    
    
    #arrival times
#     arr_times = browser.find_elements_by_xpath("//span[@data-test-id='arrival-time']")
#     arr_times_list = [value.text for value in arr_times]
#     #airline name
#     airlines = browser.find_elements_by_xpath("//span[@data-test-id='airline-name']")
#     airlines_list = [value.text for value in airlines]
#     #prices
#     prices = browser.find_elements_by_xpath("//span[@data-test-id='listing-price-dollars']")
#     price_list = [value.text for value in prices]
#     #durations
#     durations = browser.find_elements_by_xpath("//span[@data-test-id='duration']")
#     durations_list = [value.text for value in durations]
#     #stops
#     stops = browser.find_elements_by_xpath("//span[@class='number-stops']")
#     stops_list = [value.text for value in stops]
#     #layovers
#     layovers = browser.find_elements_by_xpath("//span[@data-test-id='layover-airport-stops']")
#     layovers_list = [value.text for value in layovers]
#     now = datetime.datetime.now()
#     current_date = (str(now.year) + '-' + str(now.month) + '-' + str(now.day))
#     current_time = (str(now.hour) + ':' + str(now.minute))
#     current_price = 'Price ' + '(' + current_date + ' at ' + current_time + ')'
    for i in range(len(room_type_list)):
        print("i inside compiledata for loop:",i)
        try:
            df.loc[i, 'room_type'] = room_type_list[i]
        except Exception as e:
            pass
        try:
            df.loc[i, 'price'] = price_list[i]
        except Exception as e:
            pass
        try:
            df.loc[i, 'room_area'] = room_area_list[i]
        except Exception as e:
            pass
#         try:
#             df.loc[i, 'duration'] = durations_list[i]
#         except Exception as e:
#             pass
#         try:
#             df.loc[i, 'stops'] = stops_list[i]
#         except Exception as e:
#             pass
#         try:
#             df.loc[i, 'layovers'] = layovers_list[i]
#         except Exception as e:
#             pass
#         try:
#             df.loc[i, str(current_price)] = price_list[i]
#         except Exception as e:
#             pass

In [60]:
# Connect to email 
def connect_mail(username, password):
    global server
    # Note that yahoo and gmail smtp servers only work if you reduce the account security level on the sending account.
    # Turn on the option named "Allow apps that use less secure sign in"
    server = smtplib.SMTP('smtp.mail.yahoo.com', 587)
    server.ehlo()
    server.starttls()
    server.login(username, password)

In [61]:
#Create message text for email
def create_msg():
    global msg
    msg = '\nCurrent Cheapest room:\n\nRoom Type: {}\nRoom Price: {}\nRoom Area: {}\n'.format(
                       cheapest_room_type,
                       cheapest_room_price,
                       cheapest_room_area)
    print(msg)

In [62]:
def send_email(msg):
    global message
    message = MIMEMultipart()
    message['Subject'] = 'Current Hotel prices'
    message['From'] = username
    message['to'] = 'manishbahety@yahoo.com'
    msgtext = MIMEText(msg, 'plain')
    message.attach(msgtext)
    server.sendmail(username, 'manishbahety@yahoo.com', message.as_string())

In [63]:
# Main routine
# Open the home page for Expedia 
link = 'https://www.expedia.com/'
browser.get(link)
# Let the page load by waiting for a few seconds
time.sleep(30)

In [64]:
#choose hotels only option
hotels_only = browser.find_element_by_xpath("//button[@id='tab-hotel-tab-hp']")
hotels_only.click()
time.sleep(3)
# Select the hotel name, checkin date and checkout date
goingto_hotel_chooser('quality inn gainesville tx')
checkin_date_chooser('05', '25', '2019')
checkout_date_chooser('05', '30', '2019')
time.sleep(5)
# Perform the search 
search()
click_top_result()
time.sleep(5)

Search Results ready!


In [65]:
# Collect the search data in a dataframe
compile_data()
print (df)
print('Data Frame Filled!')
#save values for email text
current_values = df.iloc[0]
cheapest_room_type = current_values[0]
cheapest_room_price = current_values[1]
cheapest_room_area = current_values[2]


[]
['$80', '$101', '$101', '$98', '$98', '$54', '$48', '$138', '$89', '$58', '$140', '$147', '$72', '$64', '$62', '$62', '$57', '$83', '$71', '$102', '$106', '$108', '$107', '$58', '$65', '$107', '$59', '$142', '$48', '$110', '$137', '$108', '$77', '$79', '$56', '$90', '$68', '$54', '$36', '$37', '$143', '$176', '$215', '$260', '$250', '$334', '$700']
[]
Empty DataFrame
Columns: []
Index: []
Data Frame Filled!


IndexError: single positional indexer is out-of-bounds

In [19]:
# Send email with cheapest flight detail
create_msg()
connect_mail(username,password)
send_email(msg)
print('Email sent!')

# Save the data frame to excel 
df.to_excel('rooms.xlsx')
print('Excel Created!')

NameError: name 'cheapest_dep_time' is not defined

In [22]:
# We are done. Close the browser session that had expedia open. 
browser.close()

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=74.0.3729.157)
  (Driver info: chromedriver=74.0.3729.6 (255758eccf3d244491b8a1317aa76e1ce10d57e9-refs/branch-heads/3729@{#29}),platform=Windows NT 10.0.17134 x86_64)
